# IMPORTS

In [1]:
import cv2
import face_recognition
import os
import numpy as np
from datetime import datetime
import threading
import requests
import json
from PIL import Image, ImageDraw, ImageFont
from dotenv import load_dotenv
from io import BytesIO


d:\Documents\Magic Mirror\venv\Lib\site-packages\face_recognition_models\__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


# CONFIGS

In [2]:
load_dotenv()
PATH_TO_FACES = os.getenv("PATH_TO_FACES", "known_faces")
ATTENDANCE_FILE = os.getenv("ATTENDANCE_FILE", "attendance_log.csv")
WEATHER_API_KEY = os.getenv("WEATHER_API_KEY")
CITY = os.getenv("CITY", "Taipei")
DASHBOARD_WIDTH = int(os.getenv("DASHBOARD_WIDTH", 400))

# FONTS

In [3]:
try:
    # Try to find the folder where this script is located
    SCRIPT_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    # If running in IDLE/Jupyter, use the current working directory
    SCRIPT_DIR = os.getcwd()

FONT_FILE = "JetBrainsMono-Medium.ttf"
FONT_PATH = os.path.join(SCRIPT_DIR, FONT_FILE)

# THREADING

In [4]:
class WebcamStream:
    def __init__(self, src=0):
        self.stream = cv2.VideoCapture(src)
        # Set buffer to 1 to reduce lag at the source
        self.stream.set(cv2.CAP_PROP_BUFFERSIZE, 1)
        (self.grabbed, self.frame) = self.stream.read()
        self.stopped = False

    def start(self):
        # Start a thread to read frames from the video stream
        threading.Thread(target=self.update, args=(), daemon=True).start()
        return self

    def update(self):
        while True:
            if self.stopped:
                return
            (self.grabbed, self.frame) = self.stream.read()

    def read(self):
        return self.frame

    def stop(self):
        self.stopped = True

# FUNCTIONS

In [5]:
def load_encodings(path):
    known_encodings = []
    known_names = []
    
    print(f"Loading faces from {path}...")
    
    if not os.path.exists(path):
        os.makedirs(path) # Create folder if it doesn't exist
        print(f"Created missing folder: {path}. Please add photos.")
        return [], []

    for filename in os.listdir(path):
        if filename.endswith((".jpg", ".png", ".jpeg")):
            img_path = os.path.join(path, filename)
            try:
                img = face_recognition.load_image_file(img_path)
                encodings = face_recognition.face_encodings(img)
                if encodings:
                    known_encodings.append(encodings[0])
                    name = os.path.splitext(filename)[0]
                    known_names.append(name)
                    print(f" -> Loaded: {filename}")
                else:
                    print(f" -> WARNING: No face found in {filename}")
            except Exception as e:
                print(f" -> ERROR loading {filename}: {e}")
            
    return known_encodings, known_names


In [6]:
def mark_attendance(name):
    today_str = datetime.now().strftime('%Y-%m-%d')
    
    # Create file if missing
    if not os.path.isfile(ATTENDANCE_FILE):
        with open(ATTENDANCE_FILE, 'w') as f:
            f.write('Name,Time,Date\n')

    # Check for duplicates correctly
    with open(ATTENDANCE_FILE, 'r') as f:
        lines = f.readlines()
        
    for line in lines:
        # Check if this specific line contains BOTH the name and today's date
        # We assume the format is: Name,Time,Date
        if name in line and today_str in line:
            return # Already logged today

    # Log new entry
    with open(ATTENDANCE_FILE, 'a') as f:
        now = datetime.now()
        dtString = now.strftime('%H:%M:%S')
        f.write(f'{name},{dtString},{today_str}\n')
        print(f"✅ LOGGED: {name}")

In [7]:
def get_font(size):
    """Safely loads JetBrains Mono or falls back to system default."""
    try:
        return ImageFont.truetype(FONT_PATH, size)
    except OSError:
        print(f"Warning: {FONT_FILE} not found in {SCRIPT_DIR}. Using default.")
        return ImageFont.load_default()

In [8]:
def get_weather():
    if not WEATHER_API_KEY:
        return None, None

    try:
        # We need forecast data for "chance of rain"
        url = f"http://api.weatherapi.com/v1/forecast.json?key={WEATHER_API_KEY}&q={CITY}&days=1&aqi=no"
        res = requests.get(url).json()
        
        # 1. Current Conditions
        current = res['current']
        forecast_day = res['forecast']['forecastday'][0]['day']
        
        weather_data = {
            "temp": f"{current['temp_c']}°C",
            "condition": current['condition']['text'],
            "wind": f"{current['wind_kph']} km/h",
            "uv": current['uv'],
            "humidity": current['humidity'], # int (0-100)
            "rain_chance": forecast_day['daily_chance_of_rain'] # int (0-100)
        }
        
        # 2. Get Icon
        icon_url = "https:" + current['condition']['icon']
        icon_res = requests.get(icon_url)
        icon_img = Image.open(BytesIO(icon_res.content)).convert("RGBA")
        icon_img = icon_img.resize((70, 70)) # Slightly bigger icon
        
        return weather_data, icon_img

    except Exception as e:
        print(f"Weather Error: {e}")
        return None, None

In [15]:
def draw_dashboard(name, weather_data, icon_img, company_info):
    height = 720
    width = DASHBOARD_WIDTH
    dash = np.zeros((height, width, 3), dtype=np.uint8)
    img_pil = Image.fromarray(dash)
    draw = ImageDraw.Draw(img_pil)
    
    # Colors
    c_white = (255, 255, 255)
    c_gray = (120, 120, 120)
    c_accent = (180, 255, 0) # Cyan (Humidity)
    c_blue = (255, 150, 50)  # Blue (Rain)
    c_purple = (255, 100, 180) # Purple (UV)
    c_dark = (40, 40, 40)

    # --- 1. TIME & DATE ---
    now = datetime.now()
    time_str = now.strftime("%H:%M")
    date_str = now.strftime("%A, %b %d").upper()
    time_w = get_font(60).getlength(time_str)
    x_time = (width - time_w) / 2
    date_w = get_font(16).getlength(date_str)
    x_date = (width - date_w) / 2
    draw.text((x_time, 20), time_str, font=get_font(60), fill=c_white)
    draw.text((x_date, 85), date_str, font=get_font(16), fill=c_accent)

    # --- 2. USER ID ---
    draw.line([(30, 120), (370, 120)], fill=c_dark, width=1)
    display_name = name if name != "Unknown" else "SCANNING..."
    draw.text((30, 135), "USER DETECTED:", font=get_font(12), fill=c_gray)
    draw.text((30, 150), display_name, font=get_font(24), fill=c_white)

    # --- 3. WEATHER MODULE ---
    y_start = 200
    draw.text((30, y_start), "ENVIRONMENT", font=get_font(12), fill=c_gray)
    
    if weather_data:
        # --- LEFT: Icon ---
        if icon_img:
            img_pil.paste(icon_img, (20, y_start + 15), icon_img)
        
        # --- CENTER: Temp, Condition & Wind (Aligned together) ---
        # Temperature
        draw.text((100, y_start + 20), weather_data['temp'], font=get_font(32), fill=c_white)
        # Condition
        draw.text((100, y_start + 55), weather_data['condition'][:15], font=get_font(14), fill=c_gray)
        # Wind (Now aligned right under condition)
        draw.text((100, y_start + 75), f"Wind: {weather_data['wind']}", font=get_font(14), fill=c_white)

        # --- BOTTOM ROW: Metrics (Humidity | Rain | UV) ---
        
        # Helper for small bars (Width = 90px to fit 3 across)
        def draw_mini_bar(x, y, percent, color, label, value_text):
            # Label
            draw.text((x, y), label, font=get_font(10), fill=c_gray)
            # Background Bar
            draw.rectangle([x, y + 15, x + 90, y + 20], fill=c_dark)
            # Filled Bar
            fill_width = int(90 * (percent / 100))
            if fill_width > 90: fill_width = 90
            draw.rectangle([x, y + 15, x + fill_width, y + 20], fill=color)
            # Value Text below bar
            draw.text((x, y + 25), value_text, font=get_font(12), fill=c_white)

        # 1. Humidity
        draw_mini_bar(30, y_start + 110, weather_data['humidity'], c_accent, "HUMIDITY", f"{weather_data['humidity']}%")

        # 2. Rain Chance
        draw_mini_bar(140, y_start + 110, weather_data['rain_chance'], c_blue, "RAIN", f"{weather_data['rain_chance']}%")
        
        # 3. UV Index (Scaled: 11 is max usually, so percent = uv/11 * 100)
        uv_val = weather_data['uv']
        uv_percent = (uv_val / 11) * 100
        draw_mini_bar(250, y_start + 110, uv_percent, c_purple, "UV INDEX", str(uv_val))

    else:
        draw.text((30, y_start + 30), "Weather Offline", font=get_font(16), fill=c_gray)

    # --- 4. NOTIFICATIONS & TASKS ---
    y_pos = 360 # Start drawing from here
    draw.line([(30, y_pos - 15), (370, y_pos - 15)], fill=c_dark, width=1)

    # A. NOTIFICATIONS (High Priority)
    alerts = company_info.get("alerts", [])
    tasks = company_info.get("tasks", [])

    if alerts:
        # Draw a "Warning" header
        draw.text((30, y_pos), "⚠️NOTIFICATIONS", font=get_font(12), fill=(100, 100, 255)) # Light Red
        y_pos += 25
        
        for alert in alerts:
            # Draw red background for alert
            draw.rectangle([30, y_pos, 370, y_pos + 22], fill=(0, 0, 80)) # Dark Red Bg
            draw.text((35, y_pos + 2), f"{alert}", font=get_font(14), fill=(255, 200, 200))
            y_pos += 30
    
    # B. TASKS (Checklist)
    # Add some spacing if we had alerts
    if alerts: y_pos += 20
    
    draw.text((30, y_pos), "ASSIGNED TASKS", font=get_font(12), fill=c_gray)
    y_pos += 25

    if name == "Unknown":
        draw.text((30, y_pos), "Identify to view tasks", font=get_font(14), fill=c_gray)
    elif not tasks:
        draw.text((30, y_pos), "No pending tasks.", font=get_font(14), fill=c_gray)
    else:
        for task in tasks[:8]: # Limit to 4 tasks to fit screen
            # Draw Checkbox Square
            draw.rectangle([30, y_pos + 4, 42, y_pos + 16], outline=c_accent, width=1)
            # Draw Task Text
            draw.text((50, y_pos), task, font=get_font(14), fill=c_white)
            y_pos += 30

    return np.array(img_pil)

In [10]:
def get_company_data(user_name):
    """Fetches global alerts + specific user tasks."""
    try:
        with open("company_data.json", "r") as f:
            data = json.load(f)
            
        # 1. Get Global Alerts
        global_alerts = data.get("global", {}).get("alerts", [])
        
        # 2. Get User Specific Data
        user_data = data.get(user_name, {"alerts": [], "tasks": []})
        user_alerts = user_data.get("alerts", [])
        user_tasks = user_data.get("tasks", [])
        
        # Combine
        return {
            "alerts": global_alerts + user_alerts,
            "tasks": user_tasks
        }
    except:
        return {"alerts": [], "tasks": []}

In [17]:
def main():
    # Load faces
    known_encodings, known_names = load_encodings(PATH_TO_FACES)
    
    # Initialize State
    company_info = {"alerts": [], "tasks": []}
    last_data_check = datetime.now()
    
    current_user = "Unknown"
    
    # Initial Weather Load
    weather_data, weather_icon = get_weather()
    last_weather_update = datetime.now()

    # Safety check if weather fails
    if weather_data is None:
        weather_data = {"temp": "--", "condition": "Offline", "wind": "--", "humidity": 0, "rain_chance": 0, "uv": 0}
    
    print("Starting Camera... Press 'q' to quit.")
    vs = WebcamStream(src=0).start()
    
    frame_count = 0

    while True:
        img = vs.read()
        if img is None:
            print("Failed to read camera")
            break

        # --- 1. WEATHER UPDATE (Every 30 mins) ---
        if (datetime.now() - last_weather_update).total_seconds() > 1800:
            # FIX: Use 'weather_data', not 'weather_text'
            weather_data, weather_icon = get_weather() 
            last_weather_update = datetime.now()

        # --- 2. DATA SYNC (Check JSON every 5 seconds) ---
        if (datetime.now() - last_data_check).total_seconds() > 5:
            # If Unknown, we just get global alerts. If User, we get their tasks too.
            search_name = current_user if current_user != "Unknown" else "GUEST"
            company_info = get_company_data(search_name)
            last_data_check = datetime.now()

        # --- 3. FACE RECOGNITION (Every 5th frame) ---
        if frame_count % 5 == 0 and len(known_encodings) > 0:
            imgS = cv2.resize(img, (0, 0), fx=0.25, fy=0.25)
            imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
            
            faces = face_recognition.face_locations(imgS)
            encodes = face_recognition.face_encodings(imgS, faces)

            found_name = "Unknown"
            if encodes:
                matches = face_recognition.compare_faces(known_encodings, encodes[0], tolerance=0.5)
                dists = face_recognition.face_distance(known_encodings, encodes[0])
                
                if len(dists) > 0:
                    best_match = np.argmin(dists)
                    if matches[best_match]:
                        found_name = known_names[best_match].upper()
                        mark_attendance(found_name)

            # Update State
            current_user = found_name
            # REMOVED: The old 'user_agendas' logic was deleted here to prevent crash

        frame_count += 1

        # --- 4. DRAW DASHBOARD ---
        sidebar = draw_dashboard(current_user, weather_data, weather_icon, company_info)

        # Resize Camera to match Dashboard Height (720px)
        img_resized = cv2.resize(img, (960, 720))

        # Combine
        final_view = np.hstack((img_resized, sidebar))

        cv2.imshow("Magic Mirror OS", final_view)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    vs.stop()
    cv2.destroyAllWindows()

# MAIN

In [19]:
if __name__ == "__main__":
    main()

Loading faces from known_faces...
 -> Loaded: teacher.jpg
Starting Camera... Press 'q' to quit.
